<a href="https://colab.research.google.com/github/gizelleguerra/genAI_demos/blob/main/Resume_GPT_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain Version of GPT Resume Seach Tool

## Installs

In [1]:
!python3 -m pip install --upgrade langchain faiss-cpu chromadb openai tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 877.7/877.7 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 58.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 83.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 5.5 MB/s eta 0:00:00
  Created wheel for chroma: filename=Chroma-0.2.0-py3-non

In [63]:
!pip install chromadb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.3/71.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 8.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.6/922.6 kB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 83.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 446.6 kB/s eta 0:00:00
    

In [2]:
!pip install pypdf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.5/249.5 kB 7.9 MB/s eta 0:00:00


## Imports and Google Drive Mount

In [3]:
# set up google drive 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import getpass
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

In [5]:
import argparse, json, time, datetime, openai
from pathlib import Path

In [7]:
def set_open_ai_key(env_path=None):
  #import json, os
  #from pathlib import Path
  try:
    with open(env_path, "r") as f:
        env_vars = json.load(f)
    os.environ["OPENAI_API_KEY"] = env_vars["OPENAI_API_KEY"]
    openai.api_key = os.environ["OPENAI_API_KEY"]
    #os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')
    openai.Model.list() #test a random command on the openai API
    return True
  except Exception as e:
    print(e)
  return False

## Setup

In [8]:
# setup API key
openai_env_path, openai.api_key = None, None
cwd = Path.cwd()
# resume_path = cwd / "Resumes"
# resume_path.mkdir(exist_ok=True)

openai_env_path = cwd/ "drive/MyDrive/Colab Notebooks/openai.env"
set_open_ai_key(openai_env_path)

True

## Load and Parse Resume Books

In [10]:
#cwd = Path.cwd()
#output_path = cwd / "drive/MyDrive/Colab Notebooks/Output"

In [9]:
#Set up document loader for pdf resume books
embeddings = OpenAIEmbeddings()
resume_path1 = "/content/drive/MyDrive/Colab Notebooks/resume_books/resume_book_2022.pdf"
resume_path2 = "/content/drive/MyDrive/Colab Notebooks/resume_books/GDI2022ResumeBook.pdf"

In [14]:
def load_resumes(path, skip_pages):
  loader = PyPDFLoader(path)
  pages = loader.load_and_split()
  pages_clean = pages[skip_pages:]
  return pages_clean

In [15]:
# actual resumes start on page 2 of this pdf compilation
r1 = load_resumes(resume_path1, 2)

In [19]:
# actual resumes start on page 1 of this pdf compilation
r2 = load_resumes(resume_path2, 1)

In [66]:
r1[0]

Document(page_content='YIN FU(206) 889-7382 ■fuyin1999@gmail.com■linkedin.com/in/yinfu1■https://github.com/fuyin19EDUCATIONNEW YORK UNIVERSITYNew York, NYThe Courant Institute of Mathematical SciencesM.S. in Mathematics in Finance(Sep. 2021 - Dec. 2022)●Coursework:Stochasticcalculus,derivativepricing,quantitativeportfoliotheory,riskmanagement,financialdata science and machine learning, time series analysis, interest rate modelingUNIVERSITY OF WASHINGTONSeattle, WABS in Mathematics(Sep. 2017 – Jun. 2021)●Coursework:Probability, linear algebra, numericalanalysis, statistics, ODEs and PDEs, measure theory●Honors:Magna Cum Laude (Top 3.5%), Dean’s ListEXPERIENCECHINA CONSTRUCTION BANK, NEW YORK BRANCHNew York, NYQuantitative Risk Analyst(Jun. 2022 – Aug. 2022)●Builtacountryriskpredictorleveraginglinearmodels,boosting,randomforestbasedonS&Pdataofeconomics and political factors, and achieved 87.2% in-sample and 78.9% out-of-sample accuracy●Drafted country risk report for the US collaborative

In [22]:
# combine resume books
resumes = r1+r2

In [24]:
len(resumes)

118

In [10]:
# start with one resume book
#loader = PyPDFLoader(resume_path1)
#pages = loader.load_and_split()

In [17]:
# actual resumes start on page 2 of this pdf compilation
#resumes = pages[2:]

## Chunk Resumes

In [74]:
# split the documents into chunks
#text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
#texts = text_splitter.split_documents(resumes)

In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [69]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set small chunk size, just to test.
    chunk_size = 500,
    chunk_overlap  = 10,
    length_function = len,
)

In [70]:
texts = text_splitter.split_documents(resumes)
print(texts[0])

page_content='YIN FU(206) 889-7382 ■fuyin1999@gmail.com■linkedin.com/in/yinfu1■https://github.com/fuyin19EDUCATIONNEW YORK UNIVERSITYNew York, NYThe Courant Institute of Mathematical SciencesM.S. in Mathematics in Finance(Sep. 2021 - Dec. 2022)●Coursework:Stochasticcalculus,derivativepricing,quantitativeportfoliotheory,riskmanagement,financialdata science and machine learning, time series analysis, interest rate modelingUNIVERSITY OF WASHINGTONSeattle, WABS in Mathematics(Sep. 2017 – Jun.' metadata={'source': '/content/drive/MyDrive/Colab Notebooks/resume_books/resume_book_2022.pdf', 'page': 2}


## Use Vector Stores to create embeddings and preform similarity search

to do: figure out which vector store to use - FAISS vs Chroma

In [28]:
from langchain.vectorstores import FAISS
from langchain.vectorstores import Chroma

In [29]:
faiss_index = FAISS.from_documents(texts, OpenAIEmbeddings())

In [30]:
docs = faiss_index.similarity_search("strong in math", k=8)

In [37]:
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content)
    #print(str(doc.metadata["page"]) + ":", doc.page_content[:100])

4: ●
Lead tutor sessions which aided students with their homework 
●
Facilitate activities that would engage students and enhance their creativity 
●
Strategize ways for students to comprehend math problems and reading courses 
●
22: main focus will deliberately  be on ideas and numerical  examples,  which we believe  help a lot in understanding the tools 
and building intuition.
MATH-GA 2048-001 SCIENTIFIC COMPUTING IN FINANCE
(3 Points) Spring term:  R. Lindsey and M. Sonthonnax
12: 3.Mathematical Tools. This component provides appropriate mathematical background in areas like 
stochastic calculus an d partial differen tial equations.
4.Computationa l Skills. These classes prov ide students with a broad range o f software skills in Java
2: linear algebra, numericalanalysis, statistics, ODEs and PDEs, measure theory●Honors:Magna Cum Laude (Top 3.5%), Dean’s ListEXPERIENCECHINA CONSTRUCTION BANK, NEW YORK BRANCHNew York, NYQuantitative Risk Analyst(Jun. 2022 – Aug.
11: ●Coursework:Prob

In [42]:
docs[0]

Document(page_content='●\nLead tutor sessions which aided students with their homework \n●\nFacilitate activities that would engage students and enhance their creativity \n●\nStrategize ways for students to comprehend math problems and reading courses \n●', metadata={'source': '/content/drive/MyDrive/Colab Notebooks/resume_books/GDI2022ResumeBook.pdf', 'page': 4})

## Call OpenAI to use GPT to answer questions based on Resumes

In [32]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

In [33]:
question = "Who knows statistics?"
#context = " Make a list from the documents given."

In [43]:
# test different prompts
context = " Make a table of results from the documents given with columns indicating page and source."
#context = " Make a list of the page numbers from the documents given, using the 'page' metadata, remove all duplicates from the list"
#context = " return the metadata 

In [ ]:
# make output in json with the following source, page, knows statistics 

In [44]:
query = question + context
print(query)

Who knows statistics? Make a table of results from the documents given with columns indicating page and source.


In [45]:
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff")

In [46]:
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': '\n\nPage | Source | Statistics\n-----|--------|-----------\n/content/drive/MyDrive/Colab Notebooks/resume_books/GDI2022ResumeBook.pdf | /content/drive/MyDrive/Colab Notebooks/resume_books/GDI2022ResumeBook.pdf | SPSS\n/content/drive/MyDrive/Colab Notebooks/resume_books/resume_book_2022.pdf | /content/drive/MyDrive/Colab Notebooks/resume_books/resume_book_2022.pdf | Probability and statistics, PCA & SVD, LDA & QDA, dynamic programming\n/content/drive/MyDrive/Colab Notebooks/resume_books/resume_book_2022.pdf | /content/drive/MyDrive/Colab Notebooks/resume_books/resume_book_2022.pdf | linear algebra, numericalanalysis, statistics, ODEs and PDEs, measure theory\n/content/drive/MyDrive/Colab Notebooks/resume_books/resume_book_2022.pdf | /content/'}

In [ ]:
# why wont it pick up the page metadata...

In [ ]:
## Try with Chroma

In [ ]:
# chain = load_qa_with_sources_chain(llm=OpenAI(), chain_type="stuff")
# chain({"input_documents": docs, "question": query}, return_source_documents=True)

In [ ]:
# db = DeepLake(dataset_path=dataset_path, read_only=True, embedding_function=embeddings)

# retriever = db.as_retriever()
# retriever.search_kwargs['distance_metric'] = 'cos'
# retriever.search_kwargs['k'] = 4

# qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever, return_source_documents=False)

# # What was the restaurant the group was talking about called?
# query = input("Enter query:")

# # The Hungry Lobster
# ans = qa({"query": query})

# print(ans)

In [ ]:
def qa(file, query, chain_type, k):
    # load document
    loader = PyPDFLoader(file)
    documents = loader.load()
    # split the documents into chunks
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)
    # select which embeddings we want to use
    embeddings = OpenAIEmbeddings()
    # create the vectorestore to use as the index
    db = Chroma.from_documents(texts, embeddings)
    # expose this index in a retriever interface
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})
    # create a chain to answer questions 
    qa = RetrievalQA.from_chain_type(
        llm=OpenAI(), chain_type=chain_type, retriever=retriever, return_source_documents=True)
    result = qa({"query": query})
    print(result['result'])
    return result

In [47]:
#file= "/content/drive/MyDrive/Colab Notebooks/resume_books/resume_book_2022.pdf"

In [48]:
# load document
#loader_chroma = PyPDFLoader(file)
#documents = loader_chroma.load()

In [55]:
# split the documents into chunks
#text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0)
#texts = text_splitter.split_documents(documents)

In [59]:
#texts_clean = texts[2:]

In [61]:
# select which embeddings we want to use
embeddings = OpenAIEmbeddings()

In [72]:
# create the vectorestore to use as the index
db = Chroma.from_documents(texts, embeddings)
# expose this index in a retriever interface
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [73]:
# create a chain to answer questions 
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever, return_source_documents=False)
result = qa({"query": query})
print(result['result'])



Page | Source | Knows Statistics 
1 | Data Analysis Intern | Yes 
2 | Skills & Additional Certifications | Yes 
2 | Leadership & Campus Activities | No 
3 | COVID19 Data Visualization in Python | Yes
